In [ ]:
!pip install bionumpy
!wget https://raw.githubusercontent.com/bionumpy/bionumpy/dev/example_data/ctcf_chr21-22.bed.gz
!wget https://raw.githubusercontent.com/bionumpy/bionumpy/dev/example_data/CTCF_chr21-22.wig.gz
!wget https://raw.githubusercontent.com/bionumpy/bionumpy/dev/example_data/chr21-22.chrom.sizes
!wget https://raw.githubusercontent.com/bionumpy/bionumpy/dev/example_data/ctcf_chr21-22.bam
!wget https://raw.githubusercontent.com/bionumpy/bionumpy/dev/example_data/chr21a22.gtf
!wget https://raw.githubusercontent.com/bionumpy/bionumpy/dev/example_data/CTCF_chr21-22.wig.gz

In [ ]:
import plotly.express as px
import numpy as np
import bionumpy as bnp

In [ ]:
## Example 1: Getting a read pileup from a BAM-file
# Reading a genome and reads from a bam file
genome = bnp.Genome.from_file("chr21-22.chrom.sizes")
reads = genome.read_intervals("ctcf_chr21-22.bam")
print(reads)

# Getting read pileup (stored efficiently as a RunLengthArray)
pileup = reads.get_pileup()

# We can index any region
region = pileup["chr22"][19970400:19970800]

px.line(region.to_array()).show()

In [ ]:
## Example 2: Finding mean read pileup value around transcription start sites
# Read genome, a wig read pileup and transcripts
genome = bnp.Genome.from_file("chr21-22.chrom.sizes", sort_names=True)
track = genome.read_track("CTCF_chr21-22.wig.gz", stream=True)
annotation = genome.read_annotation("chr21a22.gtf")
transcripts = annotation.transcripts

# Get transcript start locations and make windows around them
tss = transcripts.get_location('start')
windows = tss.get_windows(flank=500)

# Get mean read pileup within these windows and plot
signals = track[windows]
mean_signal = signals.mean(axis=0)
signal, = bnp.compute(mean_signal)
signal = signal.to_array()

px.line(x=np.arange(-500, 500), y=signal, title="Read pileup relative to TSS start",
        labels={"x": "Position relative to TSS start", "y": "Mean read pileup"}).show()

In [ ]:
## Example 3: Read pileup around CTCF peaks
import matplotlib.pyplot as plt
from bionumpy.io.delimited_buffers import NarrowPeakBuffer
from bionumpy.genomic_data.genomic_intervals import LocationEntry

# Read genome and peaks
genome = bnp.Genome.from_file("chr21-22.chrom.sizes")
peaks = bnp.open("ctcf_chr21-22.bed.gz", buffer_type=NarrowPeakBuffer).read()

# Create locations of peaks summits
location_entries = LocationEntry(peaks.chromosome, peaks.start + peaks.summit)
summits = genome.get_locations(location_entries)

# Create windows around summits and extract read pileup
windows = summits.get_windows(flank=200)
reads = genome.read_intervals("ctcf_chr21-22.bam", stream=False, stranded=True)

# Get mean pileup for reads with negative and positive strand
means = [reads[reads.strand == strand].get_pileup()[windows].mean(axis=0)
         for strand in '+-']
pos_mean, neg_mean = bnp.compute(*means)
plt.plot(np.arange(-200, 200), pos_mean.to_array())
plt.plot(np.arange(-200, 200), neg_mean.to_array())
plt.show()

In [ ]:
### Example 4: Read pileup relative to genomic variants
# Read genome and variants
genome = bnp.Genome.from_file("chr21-22.chrom.sizes")
variants = genome.read_locations("1000Genomes_chr21-22.vcf.gz", has_numeric_chromosomes=True)

# Get windows around variants and get read pileup in these windows
flank = 100
windows = variants.get_windows(flank=flank)
reads = genome.read_intervals("CTCF_chr21-22.wig.gz", stranded=True)
track = reads.get_pileup()
signals = track[windows]

# Get mean signal inside these windows and plot
mean_signal = signals.sum(axis=0)
signal, = bnp.compute(mean_signal)
signal = signal.to_array()
plt.plot(np.arange(-flank, flank), signal)
plt.show()